## Sentinel-1 data download using sentinelsat

In [2]:
# import required packages
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

import os

Definition of the environment variables and settings for downloads 

In [2]:
## change directory for download
os.chdir('D:\eo_data')

## Define search parameters

# get geoJSON file with region extent
pathJSONFile="//dapadfs/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/JSON_Ibague/IbagueJSON.geojson"

# set search dates
start_date = '20170401'
end_date = '20170405'

## Specify ESA scihub credentials

# store in variables
scihub_user = 'asalazarr'
scihub_pass = 'tila8sude'


Making the call to the API

In [3]:
# connect to the API
api = SentinelAPI(scihub_user, scihub_pass, 'https://scihub.copernicus.eu/dhus')

# download single scene by known product id
# api.download(<product_id>)

# search by polygon, time, and Hub query keywords
footprint = geojson_to_wkt(read_geojson(pathJSONFile))
products = api.query(footprint,
                     date = (start_date, end_date),
                     platformname = 'Sentinel-1')

# download all results from the search
api.download_all(products)

In [19]:
import os

# Get the metadata of the downloaded files
#own_files = api.download_all(products)[0]

# change the working directory to the location of files
os.chdir('D:\eo_data\Ibague')
print(os.getcwd())
# store the files list to a variable
eo_files = os.listdir(os.getcwd())
#print(eo_files)

D:\eo_data\Ibague


In [20]:
type(eo_files)#[0][:-3]

list

In [21]:
# Unzip files
import zipfile

# Check if a data folder exist
if not os.path.exists('data'):
    os.makedirs('data')
    print 'data folder' + ' was created'
# Check if the name of the data folder is in
if 'data' in eo_files:
    eo_files.remove('data')

##Todo catch error when a directory is in eo_files
for im_id in eo_files:    
    if not os.path.exists('data/'+im_id[:-3]+'SAFE'):
        print('Unzipping ' + im_id)
        zip_ref = zipfile.ZipFile(im_id, 'r')
        zip_ref.extractall('data')
        zip_ref.close()
    else:
        print(im_id[:-4] + ' was already uncompressed')

S1B_IW_GRDH_1SDV_20170602T104209_20170602T104234_005870_00A4B1_774C was already uncompressed
S1B_IW_GRDH_1SDV_20170614T104210_20170614T104235_006045_00A9D7_4B41 was already uncompressed
S1B_IW_GRDH_1SDV_20170626T104210_20170626T104235_006220_00AEED_0591 was already uncompressed
S1B_IW_GRDH_1SDV_20170708T104211_20170708T104236_006395_00B3E1_4D33 was already uncompressed
S1B_IW_GRDH_1SDV_20170720T104212_20170720T104237_006570_00B8E0_29C0 was already uncompressed
S1B_IW_GRDH_1SDV_20170801T104212_20170801T104237_006745_00BDE6_F318 was already uncompressed
S1B_IW_GRDH_1SDV_20170813T104213_20170813T104238_006920_00C2FF_99FD was already uncompressed
S1B_IW_GRDH_1SDV_20170825T104213_20170825T104238_007095_00C80F_6556 was already uncompressed
S1B_IW_GRDH_1SDV_20170906T104214_20170906T104239_007270_00CD25_4F8F was already uncompressed
S1B_IW_GRDH_1SDV_20170918T104214_20170918T104239_007445_00D249_A0FC was already uncompressed
S1B_IW_GRDH_1SDV_20170930T104215_20170930T104240_007620_00D74F_27DD wa

## Pre-processing using SNAP
The pre-processing workflow (to-be revised) is performed using SNAP Python API, snappy, and currently incudes the following steps:
1. Apply orbit
2. Speckle filtering
3. Terrain correction
4. Subset the area of interest
5. Logaritmic transformation (to dB)
6. Texture analysis

***

In [14]:
import snappy

from snappy import ProductIO
from snappy import HashMap
import shutil
import os  
import ast

from snappy import GPF

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')

In [22]:
print(eo_files)

['S1B_IW_GRDH_1SDV_20170602T104209_20170602T104234_005870_00A4B1_774C.zip', 'S1B_IW_GRDH_1SDV_20170614T104210_20170614T104235_006045_00A9D7_4B41.zip', 'S1B_IW_GRDH_1SDV_20170626T104210_20170626T104235_006220_00AEED_0591.zip', 'S1B_IW_GRDH_1SDV_20170708T104211_20170708T104236_006395_00B3E1_4D33.zip', 'S1B_IW_GRDH_1SDV_20170720T104212_20170720T104237_006570_00B8E0_29C0.zip', 'S1B_IW_GRDH_1SDV_20170801T104212_20170801T104237_006745_00BDE6_F318.zip', 'S1B_IW_GRDH_1SDV_20170813T104213_20170813T104238_006920_00C2FF_99FD.zip', 'S1B_IW_GRDH_1SDV_20170825T104213_20170825T104238_007095_00C80F_6556.zip', 'S1B_IW_GRDH_1SDV_20170906T104214_20170906T104239_007270_00CD25_4F8F.zip', 'S1B_IW_GRDH_1SDV_20170918T104214_20170918T104239_007445_00D249_A0FC.zip', 'S1B_IW_GRDH_1SDV_20170930T104215_20170930T104240_007620_00D74F_27DD.zip', 'S1B_IW_GRDH_1SDV_20171012T104215_20171012T104240_007795_00DC4C_C014.zip', 'S1B_IW_GRDH_1SDV_20171024T104215_20171024T104240_007970_00E14C_9097.zip', 'S1B_IW_GRDH_1SDV_201711

In [27]:
## Function to get help of the SNAP operators

def Op_help(op):
        op_spi = snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(op)
        print('Op name: {}'.format(op_spi.getOperatorDescriptor().getName()))
        print('Op alias: {}\n'.format(op_spi.getOperatorDescriptor().getAlias()))
        print('PARAMETERS:\n')
        param_Desc = op_spi.getOperatorDescriptor().getParameterDescriptors()
        for param in param_Desc:
            value_set = param_Desc[0].getValueSet()
            if len(value_set) == 0:
                print('{}: {}\nDefault Value: {}\n'.format(param.getName(),param.getDescription(),param.getDefaultValue()))
            else:
                print('{}: {}\nDefault Value: {}\nPossible param: {}\n'.format(param.getName(),param.getDescription(),param.getDefaultValue(),list(value_set)))

In [29]:
## Get a list of all Sentinel-1 toolbox operators

op_spi = snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpis().toString()
op_list = op_spi.split(', ')
listname = []
for op_str in op_list:
    to_add = op_str
    if op_str[0] == '[': to_add = op_str[1:]
    elif op_str[-1] == ']': to_add = op_str[:-1]
    #if to_add.split('.')[2] == 's1tbx':
    listname.append(to_add.split('$')[0])
listname.sort()
for name in listname:
    print(name)

org.csa.rstb.classification.gpf.PolarimetricClassificationOp
org.csa.rstb.classification.gpf.SupervisedWishartClassificationOp
org.csa.rstb.polarimetric.gpf.CrossChannelSNRCorrectionOp
org.csa.rstb.polarimetric.gpf.OrientationAngleCorrectionOp
org.csa.rstb.polarimetric.gpf.PolarimetricDecompositionOp
org.csa.rstb.polarimetric.gpf.PolarimetricMatricesOp
org.csa.rstb.polarimetric.gpf.PolarimetricParametersOp
org.csa.rstb.polarimetric.gpf.PolarimetricSpeckleFilterOp
org.esa.s1tbx.calibration.gpf.CalibrationOp
org.esa.s1tbx.calibration.gpf.RemoveAntennaPatternOp
org.esa.s1tbx.calibration.gpf.RemoveGRDBorderNoiseOp
org.esa.s1tbx.calibration.gpf.Sentinel1RemoveThermalNoiseOp
org.esa.s1tbx.fex.gpf.ChangeDetectionOp
org.esa.s1tbx.fex.gpf.FindImagePairOp
org.esa.s1tbx.fex.gpf.SubGraphOp
org.esa.s1tbx.fex.gpf.flood.FloodDetectionOp
org.esa.s1tbx.fex.gpf.forest.ForestAreaClassificationOp
org.esa.s1tbx.fex.gpf.forest.ForestAreaDetectionOp
org.esa.s1tbx.fex.gpf.oceantools.AdaptiveThresholdingOp
org

org.esa.s3tbx.idepix.algorithms.vgt.VgtOp
org.esa.s3tbx.idepix.algorithms.vgt.VgtPostProcessOp
org.esa.s3tbx.idepix.algorithms.viirs.ViirsClassificationOp
org.esa.s3tbx.idepix.algorithms.viirs.ViirsOp
org.esa.s3tbx.idepix.algorithms.viirs.ViirsPostProcessOp
org.esa.s3tbx.idepix.operators.CloudBufferOp
org.esa.s3tbx.idepix.operators.IdepixPostProcessingOp
org.esa.s3tbx.meris.FillAerosolOp
org.esa.s3tbx.meris.FillBandOp
org.esa.s3tbx.meris.GapLessSdrOp
org.esa.s3tbx.meris.L3ToL1Op
org.esa.s3tbx.meris.N1PatcherOp
org.esa.s3tbx.meris.RRToFRSOp
org.esa.s3tbx.meris.aerosol.AerosolMergerOp
org.esa.s3tbx.meris.aerosol.ModisAerosolOp
org.esa.s3tbx.meris.brr.CloudClassificationOp
org.esa.s3tbx.meris.brr.GaseousCorrectionOp
org.esa.s3tbx.meris.brr.LandClassificationOp
org.esa.s3tbx.meris.brr.RayleighCorrectionOp
org.esa.s3tbx.meris.brr.SmileCorrectionOp
org.esa.s3tbx.meris.brr.operator.BrrOp
org.esa.s3tbx.meris.cloud.BlueBandOp
org.esa.s3tbx.meris.cloud.CloudProbabilityOp
org.esa.s3tbx.meris.clou

In [31]:
Op_help("Collocate")
#Op_help("Cross-Correlation")

Op name: org.esa.snap.collocation.CollocateOp
Op alias: Collocate

PARAMETERS:

targetProductName: The name of the target product
Default Value: _collocated

targetProductType: The product type string for the target product (informal)
Default Value: COLLOCATED

renameMasterComponents: Whether or not components of the master product shall be renamed in the target product.
Default Value: true

renameSlaveComponents: Whether or not components of the slave product shall be renamed in the target product.
Default Value: true

masterComponentPattern: The text pattern to be used when renaming master components.
Default Value: ${ORIGINAL_NAME}_M

slaveComponentPattern: The text pattern to be used when renaming slave components.
Default Value: ${ORIGINAL_NAME}_S

resamplingType: The method to be used when resampling the slave grid onto the master grid.
Default Value: NEAREST_NEIGHBOUR



In [24]:
# 
eo_direc = 'D:/eo_data/Ibague/data/'

## Create a dictionary to read Sentinel-1 L1 GRD products
product = {}
for element in eo_files:
    product[element[:-4]] = {}

In [25]:
list(product.keys())

['S1B_IW_GRDH_1SDV_20170918T104214_20170918T104239_007445_00D249_A0FC',
 'S1B_IW_GRDH_1SDV_20170720T104212_20170720T104237_006570_00B8E0_29C0',
 'S1B_IW_GRDH_1SDV_20171012T104215_20171012T104240_007795_00DC4C_C014',
 'S1B_IW_GRDH_1SDV_20170906T104214_20170906T104239_007270_00CD25_4F8F',
 'S1B_IW_GRDH_1SDV_20171129T104214_20171129T104239_008495_00F103_C116',
 'S1B_IW_GRDH_1SDV_20170626T104210_20170626T104235_006220_00AEED_0591',
 'S1B_IW_GRDH_1SDV_20180128T104212_20180128T104237_009370_010D3E_6BBB',
 'S1B_IW_GRDH_1SDV_20171223T104214_20171223T104239_008845_00FC24_E700',
 'S1B_IW_GRDH_1SDV_20171105T104215_20171105T104240_008145_00E654_FADB',
 'S1B_IW_GRDH_1SDV_20170708T104211_20170708T104236_006395_00B3E1_4D33',
 'S1B_IW_GRDH_1SDV_20170930T104215_20170930T104240_007620_00D74F_27DD',
 'S1B_IW_GRDH_1SDV_20170614T104210_20170614T104235_006045_00A9D7_4B41',
 'S1B_IW_GRDH_1SDV_20171024T104215_20171024T104240_007970_00E14C_9097',
 'S1B_IW_GRDH_1SDV_20171211T104214_20171211T104239_008670_00F691

In [26]:
#rint(type(product))

# Define the area of interest
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
regPath = "//dapadfs/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/" + \
                      "JSON_Ibague/IbagueJSON.geojson"
geom = geojson_to_wkt(read_geojson(regPath))

# Define polarizations of interest
polarizations = ['VV', 'VH']

# create the orbit product
#product = GPF.createProduct("Apply-Orbit-File", apply_orbit, product)
#product.dispose()

for key, value in product.iteritems():    
    # Read the product
    value['GRD'] = ProductIO.readProduct(eo_direc+key+'.SAFE/manifest.safe')
    
    # Apply orbit
    param_orbit = HashMap()
    value['orbit'] = GPF.createProduct("Apply-Orbit-File", param_orbit, value['GRD'])
    
    # The following operations are specific por each polarization    
    for pol in polarizations:
        # Radiometric calibration
        param_calibration = HashMap()
        param_calibration.put('outputSigmaBand', True)
        param_calibration.put('sourceBands', 'Intensity_' + pol)
        param_calibration.put('selectedPolarisations', pol)
        param_calibration.put('outputImageScaleInDb', False)
        value['calibration_'+pol] = GPF.createProduct("Calibration", param_calibration, value['orbit'])
        
        # Terrain correction
        param_terraincor = HashMap()
        param_terraincor.put('demResamplingMethod', 'NEAREST_NEIGHBOUR')
        param_terraincor.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
        param_terraincor.put('applyRadiometricNormalization', True)
        param_terraincor.put('demName', 'SRTM 3Sec')
        param_terraincor.put('pixelSpacingInMeter', 10.0)
        param_terraincor.put('sourceBands', 'Sigma0_' + pol)
        param_terraincor.put('mapProjection', 'WGS84(DD)')
        value['terraincor_'+pol] = GPF.createProduct("Terrain-Correction", param_terraincor, value['calibration_'+pol])
        
        # Subset to area of interest
        param_subset = HashMap()
        param_subset.put('geoRegion', geom)
        param_subset.put('outputImageScaleInDb', False)
        param_subset.put('sourceBandNames', 'Sigma0_' + pol)
        value['region_'+pol] = GPF.createProduct("Subset", param_subset, value['terraincor_'+pol])
        
        # define the name of the output
        output_name = eo_direc + 'prep/' + key + "_" + pol + "_"
        
        # Write the results to files
        ProductIO.writeProduct(value['region_'+pol], output_name, 'BEAM-DIMAP')
        
        # dispose all the intermediate products
        value['calibration_'+pol].dispose()
        value['terraincor_'+pol].dispose()
        value['region_'+pol].dispose()
        
    #dispose all the intermediate products
    value['GRD'].dispose()
    value['orbit'].dispose()

# write the results
# ProductIO.writeProduct(orbit_pro, orbit_output, 'BEAM-DIMAP')
# print orbit_output + ' was written'

The next pre-processing steps need to be done for each polarization

In [18]:
for pol in polarizations:
    for key, value 

['Amplitude_VH', 'Intensity_VH', 'Amplitude_VV', 'Intensity_VV']

In [6]:

# define the calibration parameters
calibration = HashMap()
calibration.put('outputSigmaBand', True)
calibration.put('sourceBands', 'Intensity_' + 'VV')
calibration.put('selectedPolarisations', 'VV')
calibration.put('outputImageScaleInDb', False)

name = product.getName()[:32]
#calib = "C:/Users/asalazar/Documents/Ibague/" + name + "_calibrate_" + 'VV'

product = GPF.createProduct("Calibration", calibration, product)
#orbit_pro.dispose()

#ProductIO.writeProduct(calib_pro, calib, 'BEAM-DIMAP')
#print 'the calibration product was writed'
#calibrates.append(calib_pro)

### Speckle filtering 

How is it different time-series speckle filtering?

In [8]:

# define the speckle filtering parameters
specklefilter = HashMap()
specklefilter.put('outputSigmaBand', True)
specklefilter.put('sourceBandNames', 'Sigma0_' + 'VV')
specklefilter.put('filter', 'Gamma Map')

product = GPF.createProduct("Speckle-Filter", specklefilter, product)
#calib_pro.dispose()

#name = calib_pro.getName()[:32]
#speckle_output = speckle_path + name + "_spk_" + polarization
#ProductIO.writeProduct(speckle_pro, speckle_output, 'BEAM-DIMAP')
#filters.append(speckle_pro)

In [43]:
import os, re

prep_files = os.listdir(os.getcwd()+'\data\prep')
pre_VV = filter(re.compile(r'VV_\.dim').search, prep_files)
pre_VH = filter(re.compile(r'VH_\.dim').search, prep_files)
 
list(pre_VV) 
list(pre_VH)

['S1B_IW_GRDH_1SDV_20170602T104209_20170602T104234_005870_00A4B1_774C_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170614T104210_20170614T104235_006045_00A9D7_4B41_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170626T104210_20170626T104235_006220_00AEED_0591_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170708T104211_20170708T104236_006395_00B3E1_4D33_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170720T104212_20170720T104237_006570_00B8E0_29C0_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170801T104212_20170801T104237_006745_00BDE6_F318_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170813T104213_20170813T104238_006920_00C2FF_99FD_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170825T104213_20170825T104238_007095_00C80F_6556_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170906T104214_20170906T104239_007270_00CD25_4F8F_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170918T104214_20170918T104239_007445_00D249_A0FC_VV_.dim',
 'S1B_IW_GRDH_1SDV_20170930T104215_20170930T104240_007620_00D74F_27DD_VV_.dim',
 'S1B_IW_GRDH_1SDV_20171012T104215_20171012T104240_007795_00DC4C_C014_VV_.dim',
 'S1B_IW_GRDH_1SDV_20171024T104215_20171

In [46]:
## Read all the VV files and store in a list (idea: then interate into list to collocate and apply speckle filtering)
VV_l = []
for eofile in pre_VV:
    VV_l.append(ProductIO.readProduct(os.getcwd()+'/data/prep/' + eofile))
    

In [44]:
## Multitemporal speckle filtering


## First it is necessary to stack the bands using collocation

""""
Op alias: Collocate

PARAMETERS:

targetProductName: The name of the target product
Default Value: _collocated

targetProductType: The product type string for the target product (informal)
Default Value: COLLOCATED

renameMasterComponents: Whether or not components of the master product shall be renamed in the target product.
Default Value: true

renameSlaveComponents: Whether or not components of the slave product shall be renamed in the target product.
Default Value: true

masterComponentPattern: The text pattern to be used when renaming master components.
Default Value: ${ORIGINAL_NAME}_M

slaveComponentPattern: The text pattern to be used when renaming slave components.
Default Value: ${ORIGINAL_NAME}_S

resamplingType: The method to be used when resampling the slave grid onto the master grid.
Default Value: NEAREST_NEIGHBOUR
""""

param_coll = HashMap()
param_coll.put("master", mProduct)
param_coll.put("slave", sProduct)
param_coll.put('masterComponentPattern', '${ORIGINAL_NAME}_'+identity)
param_coll.put('masterComponentPattern', '${ORIGINAL_NAME}_'+identity)


# Then you can do it
param_specklefilter = HashMap()
param_specklefilter.put('sourceBandNames', [])
param_specklefilter.put('filter', 'Lee Sigma')
param_specklefilter.put()

Op alias: 

product = GPF.createProduct('Multi-Temporal-Speckle-Filter', out_file, in_product)


filter: None
Default Value: Lee Sigma

filterSizeX: The kernel x dimension
Default Value: 3

filterSizeY: The kernel y dimension
Default Value: 3

dampingFactor: The damping factor (Frost filter only)
Default Value: 2

estimateENL: None
Default Value: false

enl: The number of looks
Default Value: 1.0

numLooksStr: None
Default Value: 1

windowSize: None
Default Value: 7x7

targetWindowSizeStr: None
Default Value: 3x3

sigmaStr: None
Default Value: 0.9

anSize: The Adaptive Neighbourhood size
Default Value: 50


SyntaxError: EOL while scanning string literal (<ipython-input-44-c103cd0b1466>, line 34)

### Terrain correction

In [9]:

# define terrain correction parameters
terrain_correction = HashMap()
terrain_correction.put('demResamplingMethod', 'NEAREST_NEIGHBOUR')
terrain_correction.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
terrain_correction.put('applyRadiometricNormalization', True)
terrain_correction.put('demName', 'SRTM 3Sec')
terrain_correction.put('pixelSpacingInMeter', 10.0)
terrain_correction.put('sourceBands', 'Sigma0_' + 'VV')
terrain_correction.put('mapProjection', 'WGS84(DD)')

#terrain_correction.put('saveSigmaNought', True)
#terrain_correction.put('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')

product = GPF.createProduct("Terrain-Correction", terrain_correction, product)
#speckle_pro.dispose()

#terrain_output = terrain_path + name + "_corrected_" + polarization
#ProductIO.writeProduct(terrain_pro, terrain_output, 'BEAM-DIMAP')

### Subset area of interest

In [15]:
#def SubsetSentProduct(products,polarization,wkt,Subset_path,write_Product):

# read the polygon to make the subset
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
geom = geojson_to_wkt(read_geojson(pathJSONFile))

# subset operation parameters
subsettings = HashMap()
subsettings.put('geoRegion', geom)
subsettings.put('outputImageScaleInDb', False)
subsettings.put('sourceBandNames', 'Sigma0_' + 'VV')

product = GPF.createProduct("Subset", subsettings, product)
#terrain_pro.dispose()


#name = product.getName()[:32]
#ProductIO.writeProduct(SubsetProduct, Subset_output, 'BEAM-DIMAP')
#print 'the Subset product was writed'
#subsets.append(SubsetProduct)
#return(subsets)

### Logaritmic transformation (to dB)

In [16]:

# set log-scale transformation parameters
paramToDB = HashMap()
paramToDB.put('sourceBandNames', 'Sigma0_' + 'VV')

# make the log transformation
product = GPF.createProduct("LinearToFromdB", paramToDB, product)
#SubsetProduct.dispose()

# write the results
#dB_output = "C:/Users/asalazar/Documents/Ibague/_dB_"
#ProductIO.writeProduct(targetDB, dB_output, 'BEAM-DIMAP')


In [17]:
list(product.getBandNames())

['Sigma0_VV_db']

### Texture analysis

In [18]:

# paramaters for GLCM texture analysis
paramGLCM = HashMap()
paramGLCM.put('sourceBandNames', 'Sigma0_' + 'VV')
paramGLCM.put('windowSizeStr', '5x5')
paramGLCM.put('quantizerStr', 'Probabilistic Quantizer')
paramGLCM.put('quantizationLevelsStr', '16')
paramGLCM.put('displacement','4' )
paramGLCM.put('outputContrast','true')
paramGLCM.put('outputDissimilarity','true')
paramGLCM.put('outputHomogeneity','true')
paramGLCM.put('outputASM','true')
paramGLCM.put('outputEnergy','true')
paramGLCM.put('outputMean','true')
paramGLCM.put('outputVariance','true')
paramGLCM.put('outputCorrelation','true')

product = GPF.createProduct("GLCM", paramGLCM, product)

#GLCM_output = "C:/Users/asalazar/Documents/Ibague/_subset_VV"

#ProductIO.writeProduct(targetGCLM, GLCM_output, 'BEAM-DIMAP')


In [19]:
list(product.getBandNames())

['Sigma0_VV_db_Contrast',
 'Sigma0_VV_db_Dissimilarity',
 'Sigma0_VV_db_Homogeneity',
 'Sigma0_VV_db_ASM',
 'Sigma0_VV_db_Energy',
 'Sigma0_VV_db_MAX',
 'Sigma0_VV_db_Entropy',
 'Sigma0_VV_db_GLCMMean',
 'Sigma0_VV_db_GLCMVariance',
 'Sigma0_VV_db_GLCMCorrelation']

In [21]:
from snappy import jpy

def createProgressMonitor():
    PWPM = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
    JavaSystem = jpy.get_type('java.lang.System')
    monitor = PWPM(JavaSystem.out)
    return monitor

ProductIO.writeProduct(product, 'C:/Users/asalazar/Documents/Ibague/GLCM_output', 'BEAM-DIMAP', pm = createProgressMonitor())